In [1]:
import numpy as np
import torch
import psutil
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import time
import matplotlib.pyplot as plt
import random
import sys
import pickle
from torch.optim import SGD

In [2]:
sys.path.insert(1, '/home/ubuntu/Intents-Analysis/Analysis')
#sys.path.insert(1, '/Users/manjugupta/Desktop/CMU_Courses/Intents/getting_intents/Analysis')

In [3]:
from get_vocab import load_data, get_vocab
from get_frequency import get_frequency

In [4]:
#Check if cuda is available
cuda = torch.cuda.is_available()
print('CUDA is', cuda)

num_workers = 8 if cuda else 0

print(num_workers)

CUDA is True
8


In [5]:
##Needed Functions
def load_data(filename):
    a_file = open(filename, "rb")
    output = pickle.load(a_file)
    a_file.close()
    return output


def create_vocabulary(train_file):
    '''This function creates an indexed vocabulary dictionary from the training file'''
    
    vocab, _ = get_vocab(1, train_file)
    
    phone_to_idx = {'unk': 1}#Padding indx = 0, unkown_idx = 1, indexing starts from 2
    for i, phone in enumerate(vocab):
        phone_to_idx[phone] = i + 2
        
    return phone_to_idx

In [6]:
class MyDataset(Dataset):
    def __init__(self, data_file, intent_labels, phone_to_idx):
        data = load_data(data_file)
        self.all_data = []
        
        for intent in data:
            for utterance in data[intent]:
                utterance_to_idx = []
                
                for phone in utterance:
                    if phone not in phone_to_idx:
                        phone = 'unk'
    
                    utterance_to_idx.append(phone_to_idx[phone])
                
                self.all_data.append([utterance_to_idx, intent_labels[intent]])
            
    def __len__(self):
        return len(self.all_data)

    def __getitem__(self,index):
        input_vector = self.all_data[index][0]
        label = self.all_data[index][1]

        return input_vector, label

In [7]:
def collate_indic(tuple_lst):

    x_lst = [x[0] for x in tuple_lst]
    y_lst = [x[1] for x in tuple_lst]

    # collate x
    B = len(tuple_lst)#Number of training samples
    T = max(len(x) for x in x_lst)#Max length of a sentence

    # x values
    x = torch.zeros([B, T], dtype=torch.int64)
    lengths = torch.zeros(B, dtype=torch.int64)

    for i, x_np in enumerate(x_lst):
        lengths[i] = len(x_np)
        x[i,:len(x_np)] = torch.tensor(x_np)

    # collate y
    y = torch.zeros([B, 6])
    for i, y_label in enumerate(y_lst):
        y[i][y_label] = 1
        
    ids = torch.argsort(lengths, descending=True)

    return x[ids], lengths[ids], y[ids]

In [8]:
#Defining constants and labels
intent_labels = {'movie-tickets':0, 'auto-repair':1, 'restaurant-table':2, 'pizza-ordering':3, 'uber-lyft':4, 'coffee-ordering':5}
train_language = 'hindi'
test_language = 'hindi'

#Loading data
train_file = '/home/ubuntu/Intents-Analysis/TaskMasterData/Get_Phones_Combos/1_lang_train_split/taskmaster_training_' + train_language + '.pkl'
test_file = '/home/ubuntu/Intents-Analysis/TaskMasterData/Get_Phones_Combos/1_language/taskmaster_testing_' + test_language + '.pkl'

#create vocabulary and phone_to_idx
phone_to_idx = create_vocabulary(train_file)
print(len(phone_to_idx))

62


In [9]:
train_dataset = MyDataset(train_file, intent_labels, phone_to_idx)
train_loader_args = dict(shuffle=True, batch_size=128, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=True, batch_size=32)
train_loader = DataLoader(train_dataset, **train_loader_args, collate_fn=collate_indic)

test_dataset = MyDataset(test_file, intent_labels, phone_to_idx)
test_loader_args = dict(shuffle=False, batch_size=128, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=False, batch_size=1)
valid_loader = DataLoader(test_dataset, **test_loader_args, collate_fn=collate_indic)

In [17]:
class RNNClassifier(nn.Module):
    def __init__(self, vocab_size=63, embed_size=128, hidden_size=128, label_size=6):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)

        self.cnn  = nn.Conv1d(embed_size, embed_size, kernel_size=3, padding=1)
        self.cnn2 = nn.Conv1d(embed_size, embed_size, kernel_size=5, padding=2)

        self.batchnorm = nn.BatchNorm1d(embed_size*2)

        self.lstm = nn.LSTM(embed_size*2, hidden_size, num_layers=2)
        self.linear = nn.Linear(hidden_size, label_size)

    def forward(self, x, lengths):
        """
        padded_x: (B,T) padded LongTensor
        """

        # B,T,H
        input = self.embed(x)

        # (B,T,H) -> (B,H,T)
        input = input.transpose(1,2)
        embeddings = input##saved for attention

        cnn_output = torch.cat([self.cnn(input), self.cnn2(input)], dim=1)

        # (B,H,T)
        input = F.relu(self.batchnorm(cnn_output))

        input = input.transpose(1,2)

        pack_tensor = nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=True)

        _, (hn, cn) = self.lstm(pack_tensor)
        
        #doing attention
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        batch_size = hn[-1].shape[0]
        emb_size = embeddings.shape[1]
        seq_len = embeddings.shape[2]

        attention = torch.zeros([batch_size, seq_len]).to(device)
        
        
        for t in range(seq_len):
            attention[:,t] = torch.diagonal(torch.matmul(embeddings[:,:,t], hn[-1].T))
            #attention[:,t] = cos(embeddings[:,:,t], hn[-1])
            
        #normalize = torch.sum(attention, dim=1).reshape(-1,1)
        #attention = attention/normalize
        attention /= (128**0.5)

        
        output = torch.zeros([batch_size, emb_size]).to(device)
        for t in range(seq_len):
            weights = attention[:,t].reshape(-1,1)
            output += weights*embeddings[:,:,t]
        

        #output, _ = nn.utils.rnn.pad_packed_sequence(output, batch_first=True)
        #output = torch.cat([hn[0], hn[1]], dim=1)
        logits = self.linear(output)

        return logits

In [26]:
model = RNNClassifier()
#opt = optim.Adam(model.parameters(), lr = 0.001)
opt = SGD(model.parameters(), lr=0.1)
device = torch.device("cuda" if cuda else "cpu")
model.to(device)

RNNClassifier(
  (embed): Embedding(63, 128)
  (cnn): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (cnn2): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(2,))
  (batchnorm): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(256, 128, num_layers=2)
  (linear): Linear(in_features=128, out_features=6, bias=True)
)

In [27]:
opt = optim.Adam(model.parameters(), lr = 0.0001)

In [ ]:
print(train_language, test_language)
max_acc = 0

for i in range(1000):
    #print("epoch ", i)
    loss_accum = 0.0
    batch_cnt = 0

    acc_cnt = 0
    err_cnt = 0

    model.train()
    start_time = time.time()
    for batch, (x, lengths, y) in enumerate(train_loader):

        x = x.to(device)
        lengths = lengths.to(device)
        y = y.to(device)
        opt.zero_grad()
        
        logits = model(x, lengths)

        loss = nn.BCEWithLogitsLoss()(logits, y)
        loss_score = loss.cpu().item()

        loss_accum += loss_score
        batch_cnt += 1
        loss.backward()
        opt.step()

        out_val, out_indices = torch.max(logits, dim=1)
        target_val, tar_indices = torch.max(y, dim=1)

        for i in range(len(out_indices)):
            if out_indices[i] == tar_indices[i]:
                acc_cnt += 1
            else:
                err_cnt += 1

    print("train acc: ", acc_cnt/(err_cnt+acc_cnt), " train loss: ", loss_accum / batch_cnt, '--time:', time.time() - start_time)

    model.eval()
    acc_cnt = 0
    err_cnt = 0

    #start_time = time.time()
    for x, lengths, y in valid_loader:

        x = x.to(device)
        lengths = lengths.to(device)
        y = y.to(device)
        
        logits = model(x, lengths)

        out_val, out_indices = torch.max(logits, dim=1)
        target_val, tar_indices = torch.max(y, dim=1)

        for i in range(len(out_indices)):
            if out_indices[i] == tar_indices[i]:
                acc_cnt += 1
            else:
                err_cnt += 1

    current_acc = acc_cnt/(err_cnt+acc_cnt)
    if current_acc > max_acc:
        max_acc = current_acc
                
    print("validation: ", current_acc, '--max', max_acc, '--time:', time.time() - start_time)

hindi hindi
train acc:  0.218144750254842  train loss:  0.7995624516321265 --time: 14.550006628036499
validation:  0.16 --max 0.16 --time: 15.205307960510254
train acc:  0.23581379544682296  train loss:  0.5176900508611099 --time: 8.637072086334229
validation:  0.19666666666666666 --max 0.19666666666666666 --time: 9.053658246994019
train acc:  0.25042473666326875  train loss:  0.4769960862138997 --time: 12.892724752426147
validation:  0.20666666666666667 --max 0.20666666666666667 --time: 13.677891731262207
train acc:  0.2762487257900102  train loss:  0.46165764720543573 --time: 14.754877090454102
validation:  0.22333333333333333 --max 0.22333333333333333 --time: 15.454407215118408
train acc:  0.290859667006456  train loss:  0.46193427106608514 --time: 14.98074984550476
validation:  0.24333333333333335 --max 0.24333333333333335 --time: 15.816989421844482
train acc:  0.2983350322799864  train loss:  0.44643122605655505 --time: 13.691144704818726
validation:  0.25333333333333335 --max 0.2

train acc:  0.9347604485219164  train loss:  0.06628985913551372 --time: 13.109495401382446
validation:  0.8033333333333333 --max 0.81 --time: 13.621591091156006
train acc:  0.9354400271831464  train loss:  0.06569252632882285 --time: 13.269471883773804
validation:  0.8166666666666667 --max 0.8166666666666667 --time: 13.875720024108887
train acc:  0.928644240570846  train loss:  0.07233656359755475 --time: 13.270312786102295
validation:  0.7966666666666666 --max 0.8166666666666667 --time: 13.864251136779785
train acc:  0.9283044512402311  train loss:  0.07698445297453714 --time: 12.52097225189209
validation:  0.8 --max 0.8166666666666667 --time: 13.172520637512207
train acc:  0.9395174991505266  train loss:  0.06188473591338033 --time: 12.91022515296936
validation:  0.8233333333333334 --max 0.8233333333333334 --time: 13.570988893508911
train acc:  0.9473326537546721  train loss:  0.056985205120366554 --time: 13.566781997680664
validation:  0.8166666666666667 --max 0.8233333333333334 --

In [173]:
a = torch.tensor([1,2,3,4]).reshape(-1,1)

In [174]:
b = torch.tensor([ [1,2,3], [10,20,30],[100,200,300], [1000,2000,3000]])

In [175]:
a.shape, b.shape

(torch.Size([4, 1]), torch.Size([4, 3]))

In [176]:
b

tensor([[   1,    2,    3],
        [  10,   20,   30],
        [ 100,  200,  300],
        [1000, 2000, 3000]])

In [177]:
a*b

tensor([[    1,     2,     3],
        [   20,    40,    60],
        [  300,   600,   900],
        [ 4000,  8000, 12000]])

In [178]:
b.float()/a.float()

tensor([[  1.0000,   2.0000,   3.0000],
        [  5.0000,  10.0000,  15.0000],
        [ 33.3333,  66.6667, 100.0000],
        [250.0000, 500.0000, 750.0000]])